<center> 
    <font size="6" face="Baskerville Old Face"> <h1> Item response theory </h1> </font>
</center> 
<br> <br>

<center> <br>
    <font size="10" face="Baskerville Old Face"> One-parameter logistic (1PL) </font>
        <br> <br>
        <font size="5">
        \begin{equation} 
        p(x_j = 1 | \theta_i, \beta_j) = \frac{\mathrm{e}^{(\theta_i - \beta_j)}}{1 + \mathrm{e}^{(\theta_i - \beta_j)}}
        \end{equation}
        </font>
</center>

<center> <br>
    <font size="10" face="Baskerville Old Face"> Two-parameter logistic (2PL) </font>
        <br> <br>
        <font size="5">       
        \begin{equation}
        p(x_j = 1 | \theta_i, \beta_j, \alpha_j) = \frac{\mathrm{e}^{\alpha_j(\theta_i - \beta_j)}}
                                                    {1 + \mathrm{e}^{\alpha_j(\theta_i - \beta_j)}}
        \end{equation}
        <font>
</center>

<center> <br>
    <font size="10" face="Baskerville Old Face"> Three-parameter logistic (3PL) </font>
        <br> <br>
        <font size="5">
        \begin{equation}
        p(x_j=1 | \theta_i, \beta_j, \alpha_j, \chi_j) = \chi_j + (1 - \chi_j) \frac{\mathrm{e}^{\alpha_j(\theta_i - \beta_j)}}                                                                                {1 + \mathrm{e}^{\alpha_j(\theta_i - \beta_j)}}
        \end{equation}
        </font>
</center>

<center> <font face="Baskerville Old Face"> <h1> Tipo di input </h1> </font> </center> <br>

|               | Esercizio 1 | Esercizio 2 | Esercizio 3 | 
|---------------|-------------|-------------|-------------|
|   Studente A  | Voto A1     |   Voto A2   |   Voto A3   | 
|   Studente B  | Voto B1     |   Voto B2   |   Voto B3   | 
|   Studente C  | Voto C1     |   Voto C2   |   Voto C3   | 

<center> <font face="Baskerville Old Face"> <h1> Tipo di output della funzione PLN</h1> </font> </center> <br>

|               | Esercizio 1 | Esercizio 2 | Esercizio 3 | 
|---------------|-------------|-------------|-------------|
|   Studente A  | Prob A1     |   Prob A2   |   Prob A3   | 
|   Studente B  | Prob B1     |   Prob B2   |   Prob B3   | 
|   Studente C  | Prob C1     |   Prob C2   |   Prob C3   | 

<center> <font face="Baskerville Old Face"> <h1> Tipo di output della valutazione della correttezza dei dati</h1> </font> </center> <br>

|              | Esercizio 1 | Esercizio 2 | Esercizio 3 | 
|--------------|-------------|-------------|-------------|
|  Precisione  |   Prec 1    |   Prec 2    |   Prec 3    | 

In [72]:
import pandas as pd
import numpy as np
import math

class RangeError(Exception):
    """Eccezione per input fuori dal range"""
    pass

class EvalError(Exception):
    """Eccezione per metodo evalDf senza una tabella di probabilità """
    pass

class FormatError(Exception):
    """Eccezione per file dei dati di tipo non supportato """
    pass

class Irt:
    """Classe creata per applicare algoritmi di Item Response Theory ad oggetti che rappresentano le prove d'esame,
       i quali hanno una riga per studente e le colonne rappresentano gli item su cui applicare le funzioni.
       Le intersezioni rappresentano il voto preso da uno studente per quell'esercizio."""
    
    __minVal    = 0    # minimo valore possibile per difficoltà e discriminati, 0 di default
    __maxVal    = 10   # massimo valore possibile per difficoltà e discriminati, 10 di default
    __flagValue = True # flag per impedire alzare una eccezione in evalDf se ancora non è stato eseguito un metodo di PLN
        
    def __init__(self, path):        
        """Il costruttore prende come parametro il path del file csv da analizzare."""        
        
        type = path.split(".")[-1] # trovo il formato del file prendendo la parte del path dopo l'ultimo "."
        
        if type == "csv":
            self.__df = pd.read_csv(path, index_col = 0)   # rendo la prima colonna l'indice della tabella
        elif type == "xls" or type == "xlsx":
            self.__df = pd.read_excel(path, index_col = 0)  
        else:
            raise FormatError("File '" + type +"' non supportato")
            
        self.numItems = len(self.__df.columns)           # salvo il numero di items    
        self.__dfProb = pd.DataFrame(columns=self.__df.columns, index=self.__df.index) # per semplicità creo la databella di probabilità che verrà riempita dalle funzioni PLN   
     
    def changeInterval(self, min, max):
        """Modifica l'intervallo di valori delle difficoltà e dei discriminanti di default sono (0,10), estremi compresi."""
        self.__minVal = min
        self.__maxVal = max        
    
    def __getNumInput(self,msg):
        """Prende valore in input assicurandosi che sia intero e compreso nell'intervallo fissato per difficotà e discriminati."""
        while True: # ciclo finchè non ottengo un parametro corretto
            try:
                n = float(input(msg))
                if n < self.__minVal or n > self.__maxVal:
                    raise RangeError
                break
            except ValueError:
                print("L'input non è un numero!")
            except RangeError:
                print("L'input non è compreso nell'intervallo (", self.__minVal, "," , self.__maxVal, ")!")
        return n 
        
    def __getProbInput(self,msg):
        """Prende valore in input assicurandosi che sia una percentuale compresa tra 0% e 100%."""        
        while True:
            try:
                n = float(input(msg))
                if n < 0 or n > 100:
                    raise RangeError
                break
            except ValueError:
                print("L'input non è un numero!")
            except RangeError:
                print("L'input non è una probabilità compresa tra 0 e 100!")
        return n      
       
    def __getDiff(self, n):
        """Metodo usato per ottenere la lista di difficoltà degli esercizi."""
        
        lsDiff = pd.DataFrame(columns = self.__df.columns , index = ["Difficulty"])        
        print("Insersci la difficoltà per i seguenti item, essa deve essere compresa nell'intervallo (", self.__minVal, "," , self.__maxVal, ") : \n")
        for columns in lsDiff:
            lsDiff.loc["Difficulty",columns] = self.__getNumInput(columns + ": ")
        return lsDiff 

    def __getDiscrim(self, n):
        """Metodo usato per ottenere la lista di discriminanti degli esercizi."""       
        
        lsDiscrim = pd.DataFrame(columns = self.__df.columns , index = ["Discriminant"])        
        print("Insersci il discriminante per i seguenti item, essa deve essere compresa nell'intervallo (", self.__minVal, "," , self.__maxVal, ") : \n")
        for columns in lsDiscrim:
            lsDiscrim.loc["Discriminant",columns] = self.__getNumInput(columns + ": ")
        return lsDiscrim

    def __getGuess(self, n):
        """Metodo usato per ottenere la lista di probabilità di indovinare esercizi"""    
        
        lsGuess = pd.DataFrame(columns = self.__df.columns , index = ["Guess"])        
        print("Insersci la percentuale di difficoltà per i seguenti item, essa deve essere compresa tra 0 e 100: \n")
        for columns in lsGuess:
            lsGuess.loc["Guess",columns] = self.__getProbInput(columns + ": ")/100
        return lsGuess  
    
    def __estimateAbility(self, lsDiff):
        """Metodo usato per fornire una stima delle abilità degli studenti"""   
        
        lsAb         = pd.DataFrame(index = self.__df.index , columns = ["Ability"])
        difficulties = lsDiff.loc["Difficulty"]        
        
        for index, row in self.__df.iterrows():
            ability                   = (row*difficulties).sum()/difficulties.sum() # stimo mediante media ponderata
            lsAb.loc[index,"Ability"] = round(ability, 2)
        return lsAb      
    
    def pl1(self):
        """Applica il modello PL1 dell'Item Response Theory, esso usa solo il parametro delle difficoltà""" 
        
        self.__flagValue = False
        lsDiff           = self.__getDiff(self.numItems)
        lsAb             = self.__estimateAbility(lsDiff)

        for index, row in self.__dfProb.iterrows():
            for columns in self.__dfProb:
                ability    = lsAb.loc  [index,"Ability"]
                difficulty = lsDiff.loc["Difficulty",columns]
                res        = (math.exp(ability - difficulty)) / (1 + math.exp(ability - difficulty))
                self.__dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return self.__dfProb

    def pl2(self):
        """Applica il modello PL2 dell'Item Response Theory, esso usa le difficoltà ed i discriminanti"""  
        
        self.__flagValue = False        
        lsDiff           = self.__getDiff(self.numItems)
        lsAb             = self.__estimateAbility(lsDiff)
        print()
        lsDiscrim = self.__getDiscrim(self.numItems)

        for index, row in self.__dfProb.iterrows():
            for columns in self.__dfProb:
                ability      = lsAb.loc     [index,"Ability"]
                difficulty   = lsDiff.loc   ["Difficulty",columns]      
                discriminant = lsDiscrim.loc["Discriminant",columns]
                res          = (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
                self.__dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return self.__dfProb

    def pl3(self):
        """Applica il modello PL3 dell'Item Response Theory, esso usa le difficoltà, discriminanti e 
        le probabilità di indovinare gli esercizi."""              
        
        self.__flagValue = False        
        lsDiff           = self.__getDiff(self.numItems)
        lsAb             = self.__estimateAbility(lsDiff)
        print()
        lsDiscrim        = self.__getDiscrim(self.numItems)
        print()
        lsGuess          = self.__getGuess(self.numItems)
        
        for index, row in self.__dfProb.iterrows():
            for columns in self.__dfProb:
                ability      = lsAb.loc     [index,"Ability"]
                difficulty   = lsDiff.loc   ["Difficulty",columns]      
                discriminant = lsDiscrim.loc["Discriminant",columns]            
                guessProb    = lsGuess.loc  ["Guess",columns]
                res          = guessProb + (1 - guessProb) * (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
                self.__dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return self.__dfProb    
  
    def evalDf(self):
        """Questo metodo ha lo scopo di stabilire quanto sono corretti i parametri stabiliti per il calcolo delle probabilità 
        dell'ultimo modello utilizzato, prima di applicarlo è necessario usare un algoritmo PLN per l'oggetto in questione."""
        
        if self.__flagValue:
            raise EvalError("Prima di eseguire la valutazione è necessario applicare un metodo di PLN!")
        dfCorrect = pd.DataFrame(columns=self.__df.columns, index=["Precise"])
        dfC       = self.__df.fillna(0)
        for columns in self.__df:
            dfDiff = np.array([])          
            for index, row in self.__df.iterrows():
                vote = dfC.loc[index, columns]
                votePerc     = (vote * 100)/self.__maxVal 
                plProbString = self.__dfProb.loc[index, columns]
                plProbFloat  = float(plProbString[:-1])
                dfDiff       = np.append(dfDiff, abs(votePerc - plProbFloat)) # differenza tra la probabilità e la percentuale del voto rispetto al massimo
            dfCorrect.loc["Precise", columns] = str(100 - round(dfDiff.mean(), 2)) + "%"
        return dfCorrect        
    
    def getDf(self):
        """Restituisce la tabella relativa al file csv utilizzato."""
        return self.__df        

In [68]:
test = Irt("EsameEXCEL.xlsx")
test.pl1()

Insersci la difficoltà per i seguenti item, essa deve essere compresa nell'intervallo ( 0 , 10 ) : 

Esercizio 1: 3
Esercizio 2: 4
Esercizio 3: 5
Esercizio 4: 6
Esercizio 5: 7


,Esercizio 1,Esercizio 2,Esercizio 3,Esercizio 4,Esercizio 5
Studente,,,,,
Marco,98.74%,96.64%,91.37%,79.58%,58.9%
Simone,96.64%,91.37%,79.58%,58.9%,34.52%
Davide,89.66%,76.13%,53.99%,30.15%,13.71%
Edoardo,97.54%,93.58%,84.29%,66.37%,42.07%
Alex,90.72%,78.24%,56.95%,32.74%,15.19%


In [73]:
test.evalDf()

,Esercizio 1,Esercizio 2,Esercizio 3,Esercizio 4,Esercizio 5
Precise,83.34%,82.81%,89.54%,90.51%,72.88%


In [74]:
test.getDf()

,Esercizio 1,Esercizio 2,Esercizio 3,Esercizio 4,Esercizio 5
Studente,,,,,
Marco,9,8,8,6,7
Simone,8,7,7,5,6
Davide,7,6,5,4,5
Edoardo,8,7,6,6,7
Alex,7,7,6,3,5
